In [1]:
import os, pickle, logging, pickle, joblib, sys, warnings, time
warnings.simplefilter('ignore')
from scipy import stats
import numpy as np
import pandas as pd

#import catboost as cb

from sklearn import ensemble, metrics, pipeline, preprocessing, impute, model_selection
from scipy.stats import pearsonr, spearmanr

import torch
from tabpfn import TabPFNRegressor
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNRegressor

ML_RAW_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/rawData/"
FS_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/fs_PC/"
TEST_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/tesRes/"

In [ ]:
feat_col = []
with open(os.path.join(FS_PATH, "tabPFN_rfa_cv_pc_rmse")) as f:
    for line in f:
        feat_col.append(line.strip())

raw_data = pd.read_table(os.path.join(ML_RAW_PATH, "raw_data_PC.txt"), sep="\t", index_col=0)

target_col = ['GZ-PC']
raw_data = raw_data.dropna(subset=target_col)

kf = model_selection.KFold(n_splits=5, shuffle=True,  random_state=0)
y_test_final, y_pred_final = [], []
for i, (train_index, test_index) in enumerate(kf.split(raw_data)):
    print("Round {} start...".format(i))
    start_time = time.time()
    
    data_train = raw_data.iloc[train_index].copy()
    data_test = raw_data.iloc[test_index].copy()

    scale_tool = preprocessing.StandardScaler()
    scale_tool.fit(data_train.loc[:, feat_col])
    data_train.loc[:, feat_col] = scale_tool.transform(data_train.loc[:, feat_col])
    data_test.loc[:, feat_col] = scale_tool.transform(data_test.loc[:, feat_col])

    #train_sel = data_train.sample(frac=0.8, random_state=0)
    #val_sel = data_train.drop(train_sel.index).copy()

    #X_train = train_sel[feat_col].copy()
    #y_train = train_sel[target_col].values.ravel()

    #X_val = val_sel[feat_col].copy()
    #y_val = val_sel[target_col].values.ravel()

    X_train = data_train[feat_col].copy()
    y_train = data_train[target_col].values.ravel()

    X_test = data_test[feat_col].copy()
    y_test = data_test[target_col].values.ravel()

    # Initialize CatBoostClassifier
    os.environ["TABPFN_MODEL_CACHE_DIR"] = "/data2/zhoujb/package/tabpfn_ckpt"
    # >500 ignore_pretraining_limits=True
    clf_model =  TabPFNRegressor(device="cpu", random_state=0, n_jobs=-1, ignore_pretraining_limits=True) 
    #clf_model = AutoTabPFNRegressor(max_time=100, device="cpu", random_state=0)
    # Fit model
    clf_model.fit(X_train, y_train)

    y_pred = clf_model.predict(X_test)

    y_test_final.append(y_test)
    y_pred_final.append(y_pred)

    end_time = time.time()
    print("Round {}:{}".format(i, end_time-start_time))

In [ ]:
res_df = pd.DataFrame(columns=["Model", "Times", "Score", "Type"])
for i in range(len(y_test_final)):
    ## Get score
    #score_pear = pearsonr(y_test_final[i], y_pred_final[i])[0]
    score_spear = spearmanr(y_test_final[i], y_pred_final[i])[0]
    score_rmse = metrics.root_mean_squared_error(y_test_final[i], y_pred_final[i])
    score_nrmse = score_rmse / np.std(y_test_final[i])

    #res_df.loc[len(res_df)] = ["CB_ALL", i+1, score_pear, "R"]
    res_df.loc[len(res_df)] = ["tabPFN_FS", i+1, score_spear, "R"]
    res_df.loc[len(res_df)] = ["tabPFN_FS", i+1, score_rmse, "RMSE"]
    res_df.loc[len(res_df)] = ["tabPFN_FS", i+1, score_nrmse, "NRMSE"]

with open(os.path.join(TEST_RES_PATH, "PC_tabPFN_FS.pickle"), "wb") as out_f:
    pickle.dump(res_df, out_f)

In [2]:
with open(os.path.join(TEST_RES_PATH, "PC_tabPFN_FS.pickle"), "rb") as f:
    res_df = pickle.load(f)

res_df

,Model,Times,Score,Type
0,tabPFN_FS,1,0.437088,R
1,tabPFN_FS,1,8.595232,RMSE
2,tabPFN_FS,1,0.847336,NRMSE
3,tabPFN_FS,2,0.335773,R
4,tabPFN_FS,2,10.221526,RMSE
5,tabPFN_FS,2,0.946408,NRMSE
6,tabPFN_FS,3,0.645131,R
7,tabPFN_FS,3,11.600368,RMSE
8,tabPFN_FS,3,0.809011,NRMSE
9,tabPFN_FS,4,0.334598,R


In [3]:
res_df.groupby(["Type"])["Score"].mean()

Type
NRMSE     0.879492
R         0.442082
RMSE     10.479444
Name: Score, dtype: float64